<a href="https://colab.research.google.com/github/sahidmiddaempinf/product-recommendation-rag/blob/main/Step_1_Product_Recommendation_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini As A Judge for RAG Evals

## The RAG

### 1. Load the datasets

In [11]:
import json
import pandas as pd

# Update the file path to your products.json file:
file_path = 'products.json'

# Load the JSON file
with open(file_path, 'r') as file:
    products_data = json.load(file)

# Convert the products list to a DataFrame for easy manipulation
products_df = pd.DataFrame(products_data['products'])
print("Loaded products:", products_df.shape[0])
products_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'products.json'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### Packages

In [ ]:
%%capture
!pip install qdrant-client[fastembed] google-genai pandas tqdm


### Imports

In [ ]:
import pandas as pd
import json
import os
import time
from tqdm import tqdm
from google import genai
from google.genai import types
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import uuid

from google.colab import userdata

### Helpers

In [ ]:
collection_name = "product_reviews"

In [ ]:
GEMINI_KEY = userdata.get('GEMINI_API_KEY')
gemini_client = genai.Client(
    api_key=GEMINI_KEY
)

In [ ]:
def getGeminiResponse(prompt, max_tokens=8192, response_type="text/plain"):
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(
                    text=prompt
                ),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        top_p=0.95,
        top_k=40,
        max_output_tokens=max_tokens,
        response_mime_type=response_type,
    )
    response = gemini_client.models.generate_content(
        model="gemini-2.0-flash", contents=contents, config=generate_content_config
    )
    return response.text

In [5]:
getGeminiResponse("What is 2+3?")

NameError: name 'getGeminiResponse' is not defined

### EDA

In [ ]:
df = pd.read_json('reviews.json')

In [ ]:
df.head()

### Connect Qdrant

In [ ]:
QDRANT_URL = "https://qdrant-1.sg-1.cloudtop.dev"
QDRANT_KEY = userdata.get('PERSONAL_QDRANT_KEY')

In [6]:
qdrant_client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_KEY, port=None)

NameError: name 'QdrantClient' is not defined

### Create Documents

In [ ]:
documents = []
metadatas = []
ids = []

In [7]:
for index, row in df.iterrows():
    product_title = row['product_title']
    product_description = row['product_description']
    review = row['review']

    # Combine product information and review into a single document
    document = f"Title: {product_title}\nDescription: {product_description}\nReview: {review}"

    # Create metadata dictionary
    metadata = {
        "product_title": product_title,
        "product_description": product_description,
        "review": review,
        "index": index, # add index for reference
    }

    # Generate a unique ID
    doc_id = str(uuid.uuid4())

    documents.append(document)
    metadatas.append(metadata)
    ids.append(doc_id)

NameError: name 'df' is not defined

In [ ]:
len(documents)

### Load data into qdrant

In [ ]:
%%capture

qdrant_client.add(
    collection_name=collection_name,
    documents=documents,
    metadata=metadatas,
    ids=ids
)

### Test Retrieval

In [ ]:
user_query = "What are the key features of the heavy-duty workbench?"

In [ ]:
search_result = qdrant_client.query(collection_name=collection_name, query_text=user_query)

In [ ]:
for result in search_result:
    print(result.document)
    print("---")

In [ ]:
def getRagResponse(question):
    search_result = qdrant_client.query(collection_name=collection_name, query_text=question)
    system_prompt = """
      You are an intelligent assistant designed to provide accurate and informative answers based on retrieved documents.

      Your primary task is to:

      Understand the user's query.
      Retrieve relevant information from the provided context (documents).
      Synthesize the retrieved information into a coherent and accurate response.

      documents:

      """

    documents_text = ""

    doc_count = 1
    for result in search_result:
      documents_text += str(doc_count) + ": \n" + result.document + "\n\n"
      doc_count += 1

    users_query = "\n\n The user is asking: " + question

    prompt = system_prompt + documents_text + users_query

    response = getGeminiResponse(prompt)

    return response

### Test RAG

In [ ]:
getRagResponse(user_query)